## 4.3 過擬合與欠擬合
ML的根本問題是優化( optimization )以及泛化( generalization )；優化是指，調節模型已在訓練數據上得到最佳性能(即，學習算法)，而泛化是指，訓練好的模型在前所未見的數據上的性能好壞。雖然我們的目的是得到較好的泛化，但我們無法控制，只能基於訓練數據調節模型，得到較佳的泛化。<br><br>

為了防止模型從訓練數據中學到錯誤或無關緊要的模式，我們有以下手段，記住，ML真正的挑戰在於泛化，而不是過擬合：
* 1.獲取更多的訓練數據：這通常是最佳的解決方法，因為模型的訓練數據越多，泛化能力自然越好。
* 2.減少網路大小：減少模型中可學習的參數個數即可防止過擬合狀況發生，一般作法是初始化較少的層合參數，然後逐漸增加層的大小和增加新的層(以驗證集來評估)。
* 3.添加權重正規化：限制模型權重只能取較小的值(目的得到熵值更小的NN)，從而限制模型複雜度，如此可降低過擬合。其做法為將網路添加與權重值相關的「成本(cost)」，這個成本有兩種形式，如下。<br>
&nbsp;&nbsp;&nbsp;&nbsp;* L1正則化(L1 regularization)：添加的成本與權重係數的「絕對值」[權重的L1范數(norm)]成正比。<br>
&nbsp;&nbsp;&nbsp;&nbsp;* L2正則化(L2 regularization)：添加的成本與權重係數的「平方」[權重的L2范數(norm)]成正比。L2正則化也稱為權重衰減(weight decay)。<br>
* 4.dropout正規化：在訓練過程中隨機將該層的一些輸入特徵「捨棄」(設置為0)。被設置為0的特徵所佔的比例，通常在0.2~0.5之間。 <br><br>

以下將分述上面介紹的方法2.~4.：


In [1]:
# 4-3 原始模型(方法2.)
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


### 4-4 容量更小的模型(方法2.)
model = models.Sequential()<br>
model.add(layers.Dense(4, activation='relu', input_shape=(10000,)))<br>
model.add(layers.Dense(4, activation='relu'))<br>
model.add(layers.Dense(1, activation='sigmoid'))<br><br>

下圖比較了原始網路與更筱網路的驗證損失(更小的驗證損失對應更好的模型)。<br>
![4-4](../img/4-4.PNG)


### 容量更大的模型(方法2.)
model = models.Sequential()<br>
model.add(layers.Dense(512, activation='relu', input_shape=(10000,)))<br>
model.add(layers.Dense(512, activation='relu'))<br>
model.add(layers.Dense(1, activation='sigmoid'))<br><br>

為了好玩，我們也比較了一個容量更大的網路，如下。<br>
![4-5](../img/4-5.PNG)<br>
![4-6](../img/4-6.PNG)<br> 

我們可以得知，過擬合越嚴重，其驗證損失的波動也就越大。


### 4-6 向模型添加L2權重正則化(權重衰減)(方法3.)
from keras import regularizers<br>

model = models.Sequential()<br>
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                       activation='relu', input_shape=(10000,)))<br>
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                       activation='relu'))<br>
model.add(layers.Dense(1, activation='sigmoid'))<br>

l2(0.001)的意思是該層權重矩陣的每個係數都會使網路總損失增加0.001 * weight_coefficient_value，注意，這個懲罰項只有在「訓練時增加」，所以這個網路的訓練總損失會比測試損失大很多。下圖所示，此方法防止過擬合，在相同參數個數的模型下。<br>
![4-7](../img/4-7.PNG)


In [ ]:
# 4-7 Keras 中不同的權重正則化項
from keras import regularizers

regularizers.l1(0.001)                  # L1 正則化

regularizers.l1_l2(l1=0.001, l2=0.001)  # 同時做L1及L2正則化


### 4.4.3 添加dropout正則化
dropout由Geoffrey Hinton和他的學生一起開發的，其核心概念為在層的輸出值中引入噪聲(epsilon)，打破NN中不顯著的偶然模式(Hinton稱為「陰謀」)(詳細參考網路或是課本)，其中，dropout比率( dropout rate )是被設定為0的特徵所佔的比例。<br><br>

假設有一個包含某層輸出的Numpy矩陣layer_output，其形狀為(batch_size, features)。訓練時，我們隨機將矩陣中一部分值設置為0。<br><br>
layer_output *= np.random.randint(0, high=2, size=layer_output.shape) #&nbsp;&nbsp;訓練時，捨棄50%的輸出單元<br><br>

測試時，我們將輸出按照dropout比率縮小，這裡我們乘上0.5(因為前面捨棄了一半的unit)。<br><br>
layer_output *= 0.5<br><br>
注意，實戰中，亦可以讓兩個運算都在訓練時進行，而測試時輸出保持不變，如圖4-8所示。<br><br>
layer_output *= np.random.randint(0, high=2, size=layer_output.shape) #&nbsp;&nbsp;訓練時<br>
layer_output /= 0.5 #&nbsp;&nbsp;(訓練時)注意，這是成比例放大而不是比例縮小<br><br>		  
![4.4.3](../img/4.4.3.PNG)<br><br>

在Keras中，我們可以通過Dropout層向網路中引入dropoutdropout將被應用於前面一層的輸出。<br><br>
model.add(layers.Dropout(0.5))<br><br>
 
我們向IMDB網路中添加兩個Dropout層，來看一下他們降低過擬合的效果。
		 

In [2]:
# 4-8 向IMDB網路中添加dropout
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


我們將上述網路圖(圖4-9)輸出，可以得到結果如下<br>
![4.3.3.1](../img/4.4.3.1.PNG)<br><br>

總結一下，防止NN過擬合的常用方法包含：<br>
* 獲取更多的訓練數據
* 減小網路容量
* 添加權重正則化
* 添加dropout
